In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32


# ------------

torch.manual_seed(2023)

# Load the Victor Hugo dataset
with open('data/hugo_contemplations.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        # YOUR CODE
        C = n_embd
        # add you key, query and value definitions
        # define the Key layer  
        self.key = nn.Linear(C, head_size)
        # define the Query layer
        self.query = nn.Linear(C, head_size)
        # define the Value layer
        self.value =  nn.Linear(C, head_size)
        ###
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))



    def forward(self, x):
        B,T,C = x.shape
        ## YOUR CODE HERE
        # apply each layer to the input
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)
        # compute the normalize product between Q and K
        weights = q @ k.transpose(1,2) # (B, T, head_size) @ (B, 16, head_size) -> (B, T, T)
        # apply the mask (lower triangular matrix)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # apply the softmax
        weights = nn.functional.softmax(weights, dim=-1)
        out = weights @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


# generate from the model
prompt = torch.tensor(encode(['\n']))
context = torch.ones((1,1), dtype=torch.long, device=device)*prompt
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


0.009989 M parameters
step 0: train loss 4.7054, val loss 4.6731
step 500: train loss 2.7430, val loss 2.8467
step 1000: train loss 2.4996, val loss 2.5927
step 1500: train loss 2.4571, val loss 2.5604
step 2000: train loss 2.4131, val loss 2.5589
step 2500: train loss 2.3903, val loss 2.5651
step 3000: train loss 2.3752, val loss 2.5359
step 3500: train loss 2.3536, val loss 2.5113
step 4000: train loss 2.3521, val loss 2.4266
step 4500: train loss 2.3412, val loss 2.4794
step 4999: train loss 2.3423, val loss 2.4739

I O che.
.
  j'u le cha cer doralqu'acavis, sachaintarcorile reumait les, et de;
Le er! sames daient chacéce ouoisey: ffaudeut,  le lai ge Uu lars'ute,
Sansomdaint en;
I Cen s'he Et d'ont, strêteursoun jauitosonerll'ascombr veu taivit fan cesceux;
Quis jesu mass hants st mbre Pa lais su ou s meurource cai mitrevand'orure me maurspel qu'upret farbrsonus aplaries oriembetteursi t êmevanite bond'hous epranques s pral le Doile lant monnt;

 E

Jons les  vommac ple,
Bol'au to